# Hands-on: Modify attribute values with update & insert cursors

#### Learning objectives
* Differentiate between arcpy search, update, and insert cursors.
* Analyze and manipulate field data using search and update cursors.
* Construct and apply where clauses to filter records considering variations in data values.
* Add new records, including geometry, to vector datasets with insert cursors.

#### Introduction
Through this activity, you will explore the arcpy cursor capabilities for manipulating GIS data. Guided exercises will lead you through examining dataset attributes and constructing where clauses to capture variations in data values. 

You will also practice employing update cursors to standardize inconsistent data entries. Updating data records with cursors has many real-world applications.  For example, city planners might use GIS to maintain and update information about public utility networks such as water mains. An update cursor could be employed to edit attributes of specific segments of these networks, such as updating installation dates, material types, or maintenance records. This allows municipalities to ensure accurate and up-to-date information crucial for effective infrastructure planning, maintenance, and emergency response management.

Additionally, you will use an insert cursor to insert new data points into a shapefile, observing how these additions affect attribute tables and spatial representations in ArcGIS Pro. Ecologists monitoring wildlife habitats might use a workflow like this to add locations of rare species sightings collected with GPS to an existing geodatabase. 

By the end of this session, you will have extended your proficiency in data manipulation using arcpy cursors to help with data management challenges.

Run each cell, predicting the results as you go. View your results using ArcPro.  Respond to Q1-Q3 when you encounter them.

### Load modules and prepare data

We will need three modules in this notebook, arcpy, os, and traceback.   We also want to set the environment setting for overwriting output to True because while you experiement with the code, you will probably need to rerun some processes.  Currently, the notebook sets the workspace to C:/gispy/scratch.  If you need to use a different directory, you need to change it in the last line of the code cell below:

In [ ]:
import arcpy
import os
import traceback
arcpy.overwriteOutput = "True"
# If you don't have a C:/gispy/scratch directory,
# modify the workspace to a folder you can use for scratch work.
arcpy.env.workspace = "C:/gispy/scratch"

This notebook will edit the data.  To preserve our original configuration, we'll make a copy of the data before editing it.
The following code copies fires shapefile to our workspace to avoid editing our original data.

In [ ]:
fc_orig = "C:/gispy/data/ch02/fires.shp"
fc = os.path.basename(fc_orig)
try:
    #If a copy is already in the scratch directory, try to remove it.
    if arcpy.Exists(fc):
        arcpy.management.Delete(fc)
except:
    print("If the data already exists, proceed.")

try:
    # Try to make a scratch copy of the data so we can modify it.
    arcpy.management.Copy(fc_orig, fc)
except:
    traceback.print_exc()

### Examine the data
How many records? How many fields does it have? What fields does it have?

In [ ]:
field_objs = arcpy.ListFields(fc)
field_names = [field_obj.name for field_obj in field_objs]

n = len(field_names)//5
print(f"{fc} has {len(field_names)} fields and {arcpy.GetCount_management(fc)} records.")
print(f"The first {n} field names are: {field_names[:n]}")

Let's look at the first few columns:

In [ ]:
with arcpy.da.SearchCursor(fc, field_names[:n]) as sc:
    print(field_names[:n])
    for row in sc:
        print(row)
    del sc

And the last few columns:

In [ ]:

with arcpy.da.SearchCursor(fc, field_names[-n:]) as sc:
    print(field_names[-n:])
    for row in sc:
        print(row)
    del sc

### Define a where clause

The "Authorized" field looks inconsistent.  Let's take a closer look:

In [ ]:
with arcpy.da.SearchCursor(fc, "Authorized") as sc:
    for row in sc:
        print(row)
    del sc

The person named "David W. Crary Jr." is referenced several different ways.  Let's select those records and update them for consistency.

First, let's try to select them.  Our first attempt may not be correct.  Why does the following code throw a RuntimeError?

In [ ]:
where_clause = "Authorized LIKE %Crary"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row)
    del sc

If you said the RuntimeError was due to missing quotation marks, you are correct. Whenever we are specifying a *text* value, **inner quotes** need to be placed around the field value!

In [ ]:
where_clause = "Authorized LIKE '%Crary'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row)
    del sc

The above where_clause finds only two of the Crary records.  In fact, we can see that it's not returning any of the records that end with Jr.  The percent sign (%) acts as a wildcard.  When we look for words like '%Crary', they can start with anything, as long as they end with Crary.  To also capture words that have other endings, we can add another % behind Crary:  '%Crary%'

First, let's test our new selection to make sure it is correct.

In [ ]:
where_clause = "Authorized LIKE '%Crary%'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row) 
    del sc

Now we have *almost* all of them.  We are missing the uppercase one.  We could use a logical operator --OR.  Something like this.

"Authorized LIKE '%CRARY%' OR Authorized LIKE '%Crary%'"

Let's test this now:

In [ ]:
where_clause = "Authorized LIKE '%CRARY%' OR Authorized LIKE '%Crary%'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row)
    del sc 

That works in this case, but would miss other casings (e.g., crary).  A better, more general solution is to be case-insensitive. To do this, we can ask for an all uppercase version of authorized and compare that to CRARY.  That would look like this:

"UPPER(Authorized) LIKE '%CRARY%'"

Now let's test this one with a search cursor:

In [ ]:
where_clause = "UPPER(Authorized) LIKE '%CRARY%'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row)
    del sc

We have been printing the row tuple, instead of the value of Authorized.  To fix that, we need to index into the tuple:

In [ ]:
where_clause = "UPPER(Authorized) LIKE '%CRARY%'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(row[0])
    del sc 

### Update cursor

Phew!  Now we have them all.  Let's replace them with a simple "David Crary"

In [ ]:
where_clause = "UPPER(Authorized) LIKE '%CRARY%'"
with arcpy.da.UpdateCursor(fc, "Authorized", where_clause) as uc:
    for row in uc:
        print(f"Before: {row[0]}")
        row[0] = "David Crary"
        uc.updateRow(row)
    del uc



Check that it worked by running the following code:

In [ ]:
where_clause = "UPPER(Authorized) LIKE '%CRARY%'"
with arcpy.da.SearchCursor(fc, "Authorized", where_clause) as sc:
    for row in sc:
        print(f"After: {row[0]}") 

with arcpy.da.SearchCursor(fc, field_names[-n:]) as sc:
    print(field_names[-n:])
    for row in sc:
        print(row)
    del sc

The "EnteredByN" field has a similar probelm with J. Shields. His name is John, but his friends call him Jack.  

In [ ]:
with arcpy.da.SearchCursor(fc, "EnteredByN") as sc:
    for row in sc:
        print(row)
    del sc

(Q1) Add code in the box below to change all the Jack or John Shields to "John Shields": 

### Insert cursors

Though you can insert rows in the attribute table without specifying the geometry, they will not appear on a map.  To add points, you can use the geometry token (@).

In [ ]:
ic = arcpy.da.InsertCursor(fc, ["FireId", "SHAPE@"] )
ic

In [ ]:
myPoint = arcpy.Point(-70.1, 42.07)
myPoint

In [ ]:
newRow = (500000, myPoint)
newRow

In [ ]:
ic.insertRow(newRow)

In [ ]:
del ic

In Catalog view, browse to the scratch directory ( C:\gispy\scratch ). Select the fires data and refresh the __table__ view.   Scroll to the bottom of the table. Do you see the new record? (Q2) How many records are there in total?

Make a copy, "fires2.shp", of the modified fires data by running the following code:

In [ ]:
# Make another copy, so that previewing in 
# ArcGIS Pro does not interfere with editing the data.
fc_orig = fc
fc2 = fc.replace(".shp", "2.shp")
try:
    #If a copy is already in the scratch directory, try to remove it.
    if arcpy.Exists(fc2):
        arcpy.management.Delete(fc2)
except:
    print("If the data already exists, proceed.")

try:
    # Try to make a scratch copy of the data so we can modify it.
    arcpy.management.Copy(fc_orig, fc2)
except:
    traceback.print_exc()

(Q3) Next, you will add another point to fires2.shp.  Write code below to insert a row with *FireID* set to 123456, *FireName* set to "Foothills", and *ReportingU* set to "SHEND" at point -70.5, 41.  Be sure to use fc2 (instead of fc).

Add a map to the project and add the fires2.shp data to the map. Open the attribute table.  Select the points you just added and view the selected points on the map.

## Congratulations, master data manipulator!
You have learned how to:
* Manipulate attribute data with update cursors.
* Construct where clauses to filter specific subsets of data based on textual patterns.
* Standardize inconsistent data entries with update cursors to ensure data uniformity.
* Add new features to geographic datasets using insert cursors.
* Integrate these skills with ArcGIS Pro to visualize and verify changes made to spatial and attribute data.

These abilities empower you to effectively manage and enhance geospatial datasets, making informed decisions in GIS analysis and applications

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^<br>

### Spoiler alert! Answers appear below this line. If you haven't completed the questions, please do so first.
-------------------------
### Solutions

(Q1) Add code in the box below to change all the Jack or John Shields to "John Shields": 

In [ ]:
where_clause = "UPPER(EnteredByN) LIKE '%SHIELDS%'"
with arcpy.da.UpdateCursor(fc, "EnteredByN", where_clause) as uc:
    for row in uc:
        print(f"Before: {row[0]}")
        row[0] = "John Shields"
        uc.updateRow(row)
    del uc

(Q2) The table should have 13 records after one record is added.

(Q3) Add code to insert another row with *FireID* set to 123456, *FireName* set to "Foothills", and *ReportingU* set to "SHEND" at point -70.5, 41.

In [ ]:
ic = arcpy.da.InsertCursor(fc2, ["FireID", "FireName", "ReportingU", "SHAPE@"] )
myPoint = arcpy.Point(-70.5, 41)
newRow = (123456, "Foothills", "SHEND", myPoint)
ic.insertRow(newRow)
del ic